In [ ]:
import openmatrix as omx
import pandas as pd
import numpy as np
import yaml
from utility import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
with open('config.yaml', 'r') as file:
    params = yaml.safe_load(file)
    
_join = os.path.join
_dir = os.path.dirname
_norm = os.path.normpath

# paths
model_outputs_dir = params['model_dir']
skims_dir = _join(model_outputs_dir, "skims")
summary_outputs = params['summary_dir']
concept_id = params['concept_id']
ctramp_dir = params['ctramp_dir']
iteration = params['iteration']

concept_id = params['concept_id']
time_period_mapping = params['time_periods_mapping']
link21_purp_mapping = params['purpose_mapping']
mode_cat_mapping = params['mode_mapping']
time_periods = params['periods']
acc_egg_modes = params['access_egress_modes']

preprocess_dir = _join(ctramp_dir, '_pre_process_files')
perf_measure_columns = params['final_columns']
summary_dir = params['summary_dir']

time_periods = params['periods']
perf_measure_columns = params['final_columns']

best_path_skim_extension = params['best_path_skim_extension']

annual_transit_factor = params['annual_transit_factor']
annual_auto_factor = params['annual_auto_factor']

filename_extension = params['filename_extension']

In [ ]:
print(perf_measure_columns)

## Final Code

In [ ]:
tod_skims = pd.read_parquet(_join(preprocess_dir, 'tod_min_perceived_travel_time.parquet'))
tod_skims = tod_skims.reset_index()
tod_skims.head(3)

In [ ]:
# tod_skims.describe()

In [ ]:
time_periods

In [ ]:
# all_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster.parquet'))
# all_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster_2050_Baseline_R2_Run4.parquet'))
all_trips = pd.read_parquet(_join(preprocess_dir, 'trip_roster_2050_Baseline_R2_Run4_Transit.parquet'))
all_trips = all_trips.loc[all_trips['trip_mode'].isin([6,7,8])]
all_trips.head(0)

In [ ]:
connected_ods = {}
final_trips = []
for period in time_periods:
    tp_df = all_trips[all_trips['Period'] == period.lower()]
    tod_skims_df = tod_skims[['orig', 'dest', period.lower()]].copy()
    tod_skims_df = tod_skims_df.dropna()
    print(f'Connected ODs in {period}: {len(tod_skims_df):,}')
    connected_ods[period] = len(tod_skims_df)
    tod_skims_df.columns = ['orig_taz', 'dest_taz', 'perc_tt']
    tp_df = pd.merge(tp_df, tod_skims_df, on = ['orig_taz', 'dest_taz'], how='inner')
    final_trips.append(tp_df)

In [ ]:
df_connected_ods = pd.DataFrame({'Period': connected_ods.keys(), 'Connected_ODs': connected_ods.values()})
df_connected_ods.to_csv(_join(summary_dir,
                              'A1.1'+ '_Connected_ODs_' + concept_id + '_region' + filename_extension + '.csv'),
                        index=None)

In [ ]:
final_trips = pd.concat(final_trips)
len(final_trips)

In [ ]:
final_trips[1:3]

In [ ]:
# Correct for the fact that perceived travel time column has been magnified by a factor of 100 (its unit is minutes). 
final_trips['perc_tt'] = final_trips['perc_tt']/100

In [ ]:
summary_cols = params['description_a1.1']
summary_cols

In [ ]:
pp_df = []
reg_df = []
county_df = []
sd_df = []
tb_df = []
rdm_df = []
pp_df_rdm = []

for columns in summary_cols:

    #for period in time_periods:
    #print(period)

    df_temp = final_trips# [final_trips['Period'] == period.lower()]

    df_temp[columns+'_trips'] = df_temp[columns] * df_temp['trips']

    #regional value
    reg_df_temp1 = summarize_all_combinations(df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column=columns+'_trips')
    reg_df_temp1 = reg_df_temp1.rename(columns={'Value': columns+'_trips'})
    
    reg_df_temp2 = summarize_all_combinations(df_temp, groupby_columns=['Period', 'Income'], 
                                                  summary_column='trips')
    reg_df_temp2 = reg_df_temp2.rename(columns={'Value': 'trips'})
    reg_df_temp = pd.merge(reg_df_temp1, reg_df_temp2, on=['Period', 'Income'], how='left')
    reg_df_temp[columns] = reg_df_temp[columns+'_trips']/reg_df_temp['trips']
    reg_df_temp = reg_df_temp[['Period', 'Income', columns]]
    reg_df_temp = reg_df_temp.rename(columns={columns:'Value'})
    reg_df_temp['Population'] = 'Whole Population'
    reg_df_temp['Origin_zone'] = ''
    reg_df_temp['Dest_zone'] = ''
    reg_df_temp['Zone_ID'] = ''
    reg_df_temp['Concept_ID'] = concept_id
    reg_df_temp['Geography'] = 'Regional'
    reg_df_temp['Purpose'] = ''
    reg_df_temp['Mode'] = ''
    reg_df_temp['Total_Increment'] = ''
    reg_df_temp['Metric_ID'] = summary_cols[columns][1]
    reg_df_temp['Submetric'] = summary_cols[columns][1]+'.1'
    reg_df_temp['Description'] = 'Average ' + summary_cols[columns][0]+ ' in the region'
    reg_df_temp['Units'] = summary_cols[columns][2]
    reg_df_temp['Metric_name'] = summary_cols[columns][3]
    reg_df.append(reg_df_temp)
        
reg_df = pd.concat(reg_df)

In [ ]:
all_dfs = [reg_df]

for dfs in all_dfs:
    metric_name = '_perceived_travel_time_'
    dfs = dfs.reset_index(drop=True)
    dfs = dfs[perf_measure_columns]
    file_name = dfs['Submetric'][0]
    geography = '_' + dfs['Geography'][0].replace(' ', '_')
    dfs.to_csv(_join(summary_dir, file_name + metric_name + concept_id + geography + filename_extension + '_transit.csv'), index=None)
    print(len(dfs), file_name, dfs['Metric_name'][0], dfs['Value'].sum())